In [1]:
import geopandas as gpd
import numpy as np
import osmnx as ox
import pandas as pd

from shapely.geometry import Point

In [66]:
project_site = (33.9379368079315, -118.25653793837309)
buffer = 4023 #2.5 miles in meters

tags = {'school':{'amenity': ['school', 'kindergarten']},
        'grocery_store':{'shop': ['grocery', 'greengrocer', 'supermarket', 'convenience']},
        'library':{'amenity':['library']},
        'healthcare_facility':{'amenity':['clinic','hospital','doctors']},
        'pharmacy':{'amenity':['pharmacy']},
        'bank_or_ATM':{'amenity':['bank','atm']},
        'park':{'leisure':['park', 'dog_park','garden','nature_reserve','picnic_table','playground','pitch']},
        'social_services':{'amenity':['social_facility']},
        'religious_facility':{'amenity':['place_of_worship']},
        'community_center':{'amenity':['community_centre']}
        }

def find_amenities(point, tags, buffer):
    result = gpd.GeoDataFrame(crs='EPSG:4326')

    for name, tag_dict in tags.items():
        amenities = ox.geometries_from_point(point, tag_dict, buffer)

        # Drop unnecessary columns
        key = next(iter(tag_dict))
        columns_to_keep = ['unique_id', 'geometry', 'name', key]
        try:
            amenities = amenities[columns_to_keep]
        except KeyError:
            continue
        
        amenities.rename(columns={key:'type_of_amenity'}, inplace=True)

        # Append to the return GDF
        result = result.append(amenities)
    
    # Not sure why this is necessary 
    result.set_geometry('geometry', inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result



C:\Users\Ryan\.conda\envs\final\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [67]:
gdf = find_amenities(project_site, tags, buffer=4023)
gdf.head()

ipykernel_launcher:17: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.


,unique_id,geometry,name,type_of_amenity
0,node/358780007,POINT (-118.28313 33.93279),Ascension School,school
1,node/358780285,POINT (-118.21702 33.92529),Baptist Day School,school
2,node/358781087,POINT (-118.28730 33.95140),Harte Preparatory Intermediate School,school
3,node/358781644,POINT (-118.25035 33.92418),Carver Elementary School,school
4,node/358784931,POINT (-118.23785 33.96057),Graham Elementary School,school


In [81]:
# Convert GDF to exclusively point geometries (required to create shapefile)

coward_gdf = ox.projection.project_gdf(gdf)
coward_gdf['geometry'] = coward_gdf.centroid
coward_gdf = ox.projection.project_gdf(coward_gdf, to_crs=gdf.crs)

In [82]:
# Write the completed geodataframe to a shapefile
coward_gdf.to_file('data/lanzit_amenities/lanzit_amenities.shp')